## Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте [archive.org](https://archive.org/details/201309_foursquare_dataset_umn). Скачаем любым удобным образом архив fsq.zip с этой страницы. Нас будет интересовать файл checkins.dat. С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [36]:
import pandas as pd
import numpy as np
data = pd.read_table('checkins.dat',sep='|',names=['id','user_id','venue_id','latitude','longitude','created_at'],
                     skipinitialspace=True, skiprows=1)
print(data.shape,'\n',data.dtypes)

(1021966, 6) 
 id              int64
user_id         int64
venue_id        int64
latitude      float64
longitude     float64
created_at     object
dtype: object


In [37]:
data.dropna(inplace=True)
data = data.reset_index(drop=True)
print(data.shape)
data.head(3)

(396634, 6)


,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров. Эта задача — хороший повод познакомиться с алгоритмом **MeanShift**, описание при желании можно посмотреть в sklearn user guide. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [39]:
from sklearn.cluster import MeanShift
ms = MeanShift(bandwidth=0.1)
ms.fit(data[['latitude','longitude']])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [72]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_
cluster_counts = np.bincount(labels)
selected_centers = cluster_centers[np.argwhere(cluster_counts>15),:]
selected_centers = selected_centers[:,0]
print(cluster_centers.shape, selected_centers.shape)

(5536, 2) (1304, 2)


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [74]:
offices = np.array([[33.751277, -118.188740],[25.867736, -80.324116],[51.503016, -0.075479],[52.378894, 4.885084],[39.366487, 117.036146],[-33.868457, 151.205134]])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [80]:
from scipy.spatial.distance import cdist
dists = cdist(selected_centers, offices, 'euclidean')
dists = np.min(dists,axis=1)

In [88]:
Top20 = selected_centers[np.argsort(dists)[:20]]

In [92]:
np.savetxt('top20.txt',Top20,fmt='%10.5f',delimiter=',',newline='\n'); 
np.savetxt('offices.txt',offices,fmt='%10.5f',delimiter=',',newline='\n')

Карта (google maps) [здесь](https://drive.google.com/open?id=1ehh1dSTxcXC00UzYbl5o4UQ3mvDGQOPU&usp=sharing)